In [1]:
import cv2
import sys
import os
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm
import openslide
import seaborn as sns
import hdbscan
import libtiff
libtiff.libtiff_ctypes.suppress_warnings()
import time
from mask_to_xml import *

In [2]:
tissue_kernel_1 = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
tissue_kernel_2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

def get_otsu_mask(_img):
    _, _, v = cv2.split(cv2.cvtColor(_img, cv2.COLOR_RGB2YUV))
    v = cv2.medianBlur(v, 5)
    _, _mask = cv2.threshold(v, -1, 255, cv2.THRESH_OTSU)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=5)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=5)
    return _mask

def get_gray_mask(_img):
    v = cv2.cvtColor(_img, cv2.COLOR_RGB2GRAY)
    v = cv2.medianBlur(v, 5)
    _, _mask = cv2.threshold(v, -1, 255, cv2.THRESH_OTSU)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=2)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_OPEN, tissue_kernel_1, iterations=5)
    _mask = cv2.morphologyEx(_mask, cv2.MORPH_CLOSE, tissue_kernel_1, iterations=5)
    _mask = ~_mask
    return _mask

def get_level(n, cnt):
    if (n % 2) == 0:
        return get_level((n // 2), cnt+1)
    else:
        return cnt

def get_mask(slide_name, magnification):
    slide_path = glob(f'/workspace/data3/A100_data/*/*/{slide_name}.svs')
    if len(slide_path) == 0:
        slide_path = glob(f'/workspace/data4/A100_data/*/*/{slide_name}.svs')
        if len(slide_path) == 0:
            slide_path = glob(f'/workspace/data3/A100_data/*/{slide_name}.svs')
    slide_path = slide_path[0]
    
    wsi_slide = openslide.OpenSlide(slide_path)
    mpp = wsi_slide.properties['aperio.MPP']
    if float(mpp) < 0.40:
        ORG_MAG = 40
    else:
        ORG_MAG = 20
    
    LEVEL = get_level(int(ORG_MAG // 1.25), 0)
    w_, h_ = wsi_slide.level_dimensions[LEVEL//2]
    wsi_array = np.array(wsi_slide.read_region([0, 0], LEVEL//2, [w_, h_]))
    if LEVEL % 2:
        wsi_array = cv2.resize(wsi_array, (w_//2, h_//2))
    otsu_mask = get_otsu_mask(wsi_array)
    gray_mask = get_gray_mask(wsi_array)
    tissue_mask = cv2.bitwise_or(otsu_mask, gray_mask)
    
    LEVEL = get_level(int(ORG_MAG // magnification), 0)
    w_, h_ = wsi_slide.level_dimensions[LEVEL//2]
    if LEVEL % 2:
        w_, h_ = w_//2, h_//2
    tissue_mask = cv2.resize(tissue_mask, (w_, h_))
    
    return tissue_mask, mpp, LEVEL

In [3]:
sample = ['A100_00408_01_stm_c1.png']

In [4]:
time_list = []

for i in range(len(sample)):
    time_start = time.perf_counter()
    
    file_name = sample[i][:-4]
    tissue_mask, mpp, level = get_mask(file_name, 5)

    read_path = '/workspace/data4/changwoo/SDP/mask/Carcinoma_v4/5X/'+sample[i]
    img = cv2.imread(read_path)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_gray = cv2.bitwise_and(img_gray, tissue_mask)
    img_blur = cv2.GaussianBlur(img_gray, (0, 0), 5)
    thres = int(0.5 * 255)
    x_idx, y_idx = np.where(img_blur >= thres)
    gray_2d = np.transpose(np.array([x_idx, y_idx]))
    clusterer = hdbscan.HDBSCAN(min_cluster_size = 300, min_samples = 100, gen_min_span_tree=True)
    hdb = clusterer.fit(gray_2d)
    labels = hdb.fit_predict(gray_2d)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    contours = []
    for i in range(n_clusters):
        x = np.zeros((len(img_gray), len(img_gray[0])))
        for j in range(len(labels)):
            if labels[j] == i:
                x[gray_2d[j][0], gray_2d[j][1]] = 255
        x = x.astype(np.uint8)
        contour, hierarchy = cv2.findContours(image = x, mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_SIMPLE)
        contours.append(np.asarray(contour[0]))

    # zone size 이용
    img_copy = img.copy()
    l = []
    for i in range(1, len(contours)):
        if len(contours[i]) > 4:
            (x, y), (h, w), angle = cv2.fitEllipse(contours[i])
            if h > w:
                l.append(h)
            else:
                l.append(w)
    num = len(contours)
    max_length = max(l)
    max_length_mm = max_length * 0.002

    total_area = 0
    area = []
    for i, cnt in enumerate(contours):
        a = cv2.contourArea(cnt)
        total_area = total_area + a
        area.append(a)
    max_area = max(area)

    _, tissue_thresh = cv2.threshold(tissue_mask, 0, 255, cv2.THRESH_BINARY_INV)
    # Find the contour of the figure 
    tissue_contours, hierarchy = cv2.findContours(image = tissue_thresh, mode = cv2.RETR_TREE, method = cv2.CHAIN_APPROX_SIMPLE)
    tissue_area = 0
    for i, cnt in enumerate(tissue_contours):
        a = cv2.contourArea(cnt)
        tissue_area = tissue_area + a
    if tissue_area == 0:
        percent = 0
    else:
        percent = total_area/tissue_area*100

    time_end = time.perf_counter()
    time_list.append(time_end - time_start)
        
    final = [file_name, num, total_area, percent, max_length, max_length_mm, max_area]
    print(final)
    

CPU times: user 13 µs, sys: 6 µs, total: 19 µs
Wall time: 41.5 µs
['A100_00408_01_stm_c1', 265, 443757.5, 0.26757118085617027, 161.46766662597656, 0.3229353332519531, 10237.5]
